# 1. Introducción a las Fuentes de Error

En la resolución de problemas de ingeniería, las soluciones numéricas siempre son **aproximaciones**. Es crucial entender de dónde provienen los errores para garantizar la **confiabilidad** de nuestros modelos.

El **Error Numérico Total ($\epsilon_t$)** es la diferencia entre el valor verdadero del problema y la solución aproximada que obtenemos con la computadora.

$$\epsilon_t = \text{Valor Verdadero} - \text{Valor Aproximado}$$

## 2. Clasificación de las Fuentes de Error

| Tipo de Error | Origen | Consecuencia en el Cálculo |
| :--- | :--- | :--- |
| **Error de Planteamiento** | Simplificación del problema físico o ambiental para crear el modelo matemático. | El modelo no refleja el 100% de la realidad (Ej. Suponer $\text{k}$ constante en el ambiente). |
| **Error del Método (Truncamiento)** | Uso de un método numérico para aproximar una función o proceso matemático infinito (Ej. Expansión de Taylor, la fórmula de Euler). | Introduce un error sistemático que disminuye al reducir el tamaño del paso ($\Delta \text{t}$) o al usar métodos de mayor orden. |
| **Errores de Representación y Redondeo** | La memoria finita de la computadora (32 o 64 bits) obliga a cortar o redondear números reales (como $\pi$ o $1/3$). | Introduce pequeños errores aleatorios en cada operación aritmética, especialmente notables en precisiones bajas. |
| **Errores de Propagación** | Un error inicial (de redondeo o truncamiento) se arrastra y amplifica a través de las operaciones sucesivas de un algoritmo. | Puede hacer que un algoritmo sea **inestable** si el problema está mal condicionado. |

## 3. Métrica de Error: Residual y Error Relativo

### Error Residual
El error residual mide qué tanto la solución aproximada **satisface** la ecuación original. Si tenemos $\text{f}(\text{x}) = 0$, el residual es $R = \text{f}(\text{x}_{\text{aprox}})$. Un residual cercano a cero indica que $\text{x}_{\text{aprox}}$ es una buena raíz de la ecuación.

### Error Relativo ($\epsilon_r$)
Es la medida más útil, ya que estandariza el error absoluto.

$$\epsilon_r = \frac{|\text{Error Absoluto}|}{|\text{Valor Verdadero}|} \times 100\%$$

# 4. Ejemplo Integrador en Python: Decaimiento de Contaminantes

**Problema:** Determinar la concentración de un contaminante ($\text{C}$) después de 2 días.
**Modelo Analítico:** $\text{C}(\text{t}) = \text{C}_0 e^{-\text{kt}}$

**Parámetros:**
* Concentración Inicial ($\text{C}_0$): $10/3 \approx 3.33333333333$ mg/L
* Tasa de Decaimiento ($\text{k}$): $0.25$ $\text{días}^{-1}$
* Tiempo ($\text{t}$): $2$ días

In [ ]:
import numpy as np

# --- 1. Valores Teóricos/Verdaderos ---
# C_0 es 10/3 (un número periódico real)
C0_verdadero = 10 / 3
k_verdadero = 0.25
t_analisis = 2

# Valor Verdadero (usando alta precisión de NumPy para simular el valor 'exacto')
C_verdadero = C0_verdadero * np.exp(-k_verdadero * t_analisis)

# Impresión del Valor Verdadero
print(f"VALOR ANALÍTICO (Verdadero): {C_verdadero:.10f} mg/L\n")


# --- 2. Simulando Errores de Representación y Redondeo (Precisión Simple) ---
# Usamos np.float32 (precisión simple) para simular la limitación de la máquina
# Esto introduce un ERROR DE REDONDEO/REPRESENTACIÓN en C0.
C0_aprox_32 = np.float32(10 / 3)
k_aprox_32 = np.float32(0.25)
t_aprox_32 = np.float32(2)

# La operación completa acumula el error de redondeo y produce ERROR DE PROPAGACIÓN
C_redondeo = C0_aprox_32 * np.exp(-k_aprox_32 * t_aprox_32)

print("--- ANÁLISIS DEL CÁLCULO DIRECTO ---")
print(f"C0 (32-bit): {C0_aprox_32:.10f}")
print(f"C (32-bit, con ERROR DE REDONDEO/PROPAGACIÓN): {C_redondeo:.10f} mg/L")

# Calculamos la magnitud del Error Numérico Total aquí
error_total_abs = abs(C_verdadero - C_redondeo)
error_relativo = (error_total_abs / C_verdadero) * 100

print(f"\nError Numérico Total (Absoluto): {error_total_abs:.10f}")
print(f"Error Relativo (por Redondeo/Propagación): {error_relativo:.8f} %")


# --- 3. Error Residual (Evaluando la Ecuación) ---
# Usamos un valor aproximado hipotético (simulando una solución del método de Euler)
C_aprox_euler = 2.0315000000 # Supuesta solución obtenida por un método (ej. Euler, con ERROR DEL MÉTODO)

# La ecuación analítica es f(C) = C - C0 * exp(-kt) = 0
residual = C_aprox_euler - (C0_verdadero * np.exp(-k_verdadero * t_analisis))

print("\n--- ANÁLISIS DEL ERROR RESIDUAL ---")
print(f"Solución Aproximada (Euler, con Error del Método): {C_aprox_euler:.10f}")
print(f"Error Residual (f(C_aprox)): {residual:.10f}")

# Nota sobre el Error de Planteamiento: Este error no es cuantificable con Python,
# ya que requiere una medición de campo real vs. el modelo.
# Se discute en la celda de texto.

VALOR ANALÍTICO (Verdadero): 2.0217688657 mg/L

--- ANÁLISIS DEL CÁLCULO DIRECTO ---
C0 (32-bit): 3.3333332539
C (32-bit, con ERROR DE REDONDEO/PROPAGACIÓN): 2.0217688084 mg/L

Error Numérico Total (Absoluto): 0.0000000573
Error Relativo (por Redondeo/Propagación): 0.00000284 %

--- ANÁLISIS DEL ERROR RESIDUAL ---
Solución Aproximada (Euler, con Error del Método): 2.0315000000
Error Residual (f(C_aprox)): 0.0097311343


## 5. Conclusiones para la Clase

* **Error de Planteamiento:** No se puede eliminar. Se minimiza mejorando el modelo (ej. usando un $\text{k}(\text{T})$ en lugar de una constante).
* **Error del Método (Truncamiento):** Se puede reducir usando métodos de orden superior (Ej. Runge-Kutta en lugar de Euler) o reduciendo el paso ($\Delta \text{t}$).
* **Errores de Redondeo/Propagación:** Se pueden mitigar usando **doble precisión** (`float64` en Python/NumPy) y algoritmos **numéricamente estables**.
* **La Calidad de la Solución** se mide con el **Error Relativo** ($\epsilon_r$) y la cercanía del **Residual** a cero.
